In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import missingno as msno
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, r2_score
from sklearn.impute import KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing


import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [2]:
data = pd.read_csv('adult.csv')

In [3]:
data.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [4]:
data.replace('?', np.nan, inplace = True) # Here we replace '?' with nan
data.head(20)

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,<=50K
5,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K
6,29,NaN,227026,HS-grad,9,Never-married,NaN,Unmarried,Black,Male,0,0,40,United-States,<=50K
7,63,Self-emp-not-inc,104626,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,3103,0,32,United-States,>50K
8,24,Private,369667,Some-college,10,Never-married,Other-service,Unmarried,White,Female,0,0,40,United-States,<=50K
9,55,Private,104996,7th-8th,4,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,10,United-States,<=50K


In [5]:
data.dropna(inplace = True)

In [6]:


# Define the income mapping
income_mapping = {'<=50K': 0, '>50K': 1}

# Convert the 'Salary' column using the mapping
data['income'] = data['income'].map(income_mapping)

# Verify the conversion
print(data['income'].head())

0    0
1    0
2    1
3    1
5    0
Name: income, dtype: int64


In [7]:
def convert_marital_status(status):
    if status in ['Married-civ-spouse', 'Married-spouse-absent', 'Married-AF-spouse']:
        return 'married'
    elif status in ['Never-married', 'Separated', 'Widowed']:
        return 'single'
    else:
        return 'divorced'

data['marital-status'] = data['marital-status'].apply(convert_marital_status)

data['native-country'] = data['native-country'].replace('Outlying-US(Guam-USVI-etc)' , 'US Minor Islands')


In [8]:
data['gender'] = data['gender'].map({'Male': 1, 'Female': 0})

In [9]:
columns_to_keep = ['workclass', 'educational-num', 'marital-status', 'occupation', 'gender', 'native-country', 'income']
Features = data[columns_to_keep]

In [10]:
X= Features

X = X.drop(columns=['income'])  # i removed income columns from x
y = data['income'] # in y only income 

x_train, x_test, y_train, y_test = train_test_split(X , y, test_size = 0.3) # train test splidata

In [11]:
print(X)

          workclass  educational-num marital-status         occupation  \
0           Private                7         single  Machine-op-inspct   
1           Private                9        married    Farming-fishing   
2         Local-gov               12        married    Protective-serv   
3           Private               10        married  Machine-op-inspct   
5           Private                6         single      Other-service   
...             ...              ...            ...                ...   
48837       Private               12        married       Tech-support   
48838       Private                9        married  Machine-op-inspct   
48839       Private                9         single       Adm-clerical   
48840       Private                9         single       Adm-clerical   
48841  Self-emp-inc                9        married    Exec-managerial   

       gender native-country  
0           1  United-States  
1           1  United-States  
2           1  Uni

In [12]:
categorical_columns = X.select_dtypes(include = 'object').columns
numerical_columns = X.select_dtypes(exclude= 'object').columns

In [13]:
X

,workclass,educational-num,marital-status,occupation,gender,native-country
0,Private,7,single,Machine-op-inspct,1,United-States
1,Private,9,married,Farming-fishing,1,United-States
2,Local-gov,12,married,Protective-serv,1,United-States
3,Private,10,married,Machine-op-inspct,1,United-States
5,Private,6,single,Other-service,1,United-States
...,...,...,...,...,...,...
48837,Private,12,married,Tech-support,0,United-States
48838,Private,9,married,Machine-op-inspct,1,United-States
48839,Private,9,single,Adm-clerical,0,United-States
48840,Private,9,single,Adm-clerical,1,United-States


In [14]:
categorical_columns

Index(['workclass', 'marital-status', 'occupation', 'native-country'], dtype='object')

In [15]:
numerical_columns

Index(['educational-num', 'gender'], dtype='object')

In [16]:
for i in categorical_columns:
    print(i)
    print(data[i].unique())
    

workclass
['Private' 'Local-gov' 'Self-emp-not-inc' 'Federal-gov' 'State-gov'
 'Self-emp-inc' 'Without-pay']
marital-status
['single' 'married' 'divorced']
occupation
['Machine-op-inspct' 'Farming-fishing' 'Protective-serv' 'Other-service'
 'Prof-specialty' 'Craft-repair' 'Adm-clerical' 'Exec-managerial'
 'Tech-support' 'Sales' 'Priv-house-serv' 'Transport-moving'
 'Handlers-cleaners' 'Armed-Forces']
native-country
['United-States' 'Peru' 'Guatemala' 'Mexico' 'Dominican-Republic'
 'Ireland' 'Germany' 'Philippines' 'Thailand' 'Haiti' 'El-Salvador'
 'Puerto-Rico' 'Vietnam' 'South' 'Columbia' 'Japan' 'India' 'Cambodia'
 'Poland' 'Laos' 'England' 'Cuba' 'Taiwan' 'Italy' 'Canada' 'Portugal'
 'China' 'Nicaragua' 'Honduras' 'Iran' 'Scotland' 'Jamaica' 'Ecuador'
 'Yugoslavia' 'Hungary' 'Hong' 'Greece' 'Trinadad&Tobago'
 'US Minor Islands' 'France' 'Holand-Netherlands']


In [17]:
workclass_cat = ['Private', 'Local-gov', 'Self-emp-not-inc', 'Federal-gov',
       'State-gov', 'Self-emp-inc', 'Without-pay', 'Never-worked']


marital_status_cat = ['single' ,'married' ,'divorced']


occupation_cat = ['Machine-op-inspct' ,'Farming-fishing', 'Protective-serv' 
 ,'Other-service', 'Prof-specialty', 'Craft-repair' ,'Adm-clerical'
 ,'Exec-managerial' ,'Tech-support' ,'Sales', 'Priv-house-serv'
 ,'Transport-moving' ,'Handlers-cleaners' ,'Armed-Forces']

native_country_cat = ['United-States', 'Peru', 'Guatemala', 'Mexico','Dominican-Republic','Ireland','Germany', 'Philippines', 'Thailand', 'Haiti', 'El-Salvador','Puerto-Rico', 'Vietnam' ,'South', 'Columbia' ,'Japan', 'India', 'Cambodia','Poland', 'Laos' ,'England', 'Cuba', 'Taiwan' ,'Italy' ,
                      'Canada' ,'Portugal','China', 'Nicaragua' ,
                      'Honduras', 'Iran', 'Scotland', 'Jamaica' ,
'Ecuador','Yugoslavia' ,'Hungary' ,'Hong', 'Greece' ,'Trinadad&Tobago',
'US Minor Islands','France','Holand-Netherlands']





In [18]:

from sklearn.impute import SimpleImputer # Missing values
from sklearn.preprocessing import StandardScaler # Feature scaling 
from sklearn.preprocessing import OrdinalEncoder # To rank categorical features
# Pipeline
from sklearn.pipeline import Pipeline #To add everything together 
from sklearn.compose import ColumnTransformer # Combine everything together


Creating Numerical Pipeline

In [19]:
num_pipe = Pipeline(
    steps = [
        ('imputer', SimpleImputer(strategy= 'median')), 
        ('scaler', StandardScaler())
    ]
)

In [20]:
cat_pipe = Pipeline(
    steps = [
        ('imputer', SimpleImputer(strategy='most_frequent')), 
        ('ordinalencoder', OrdinalEncoder(categories=[workclass_cat, marital_status_cat, occupation_cat,native_country_cat])), 
        ('scaler', StandardScaler())
    ]

)


Combine both the pipelines

In [21]:

preprocessor = ColumnTransformer([
    ('num_pipe', num_pipe, numerical_columns), 
    ('cat_pipe', cat_pipe, categorical_columns)
])

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.25, random_state= 45)

In [23]:
X_train = pd.DataFrame(preprocessor.fit_transform(X_train), columns=preprocessor.get_feature_names_out()) 
X_test = pd.DataFrame(preprocessor.transform(X_test), columns = preprocessor.get_feature_names_out())

In [24]:
X_train.head()

,num_pipe__educational-num,num_pipe__gender,cat_pipe__workclass,cat_pipe__marital-status,cat_pipe__occupation,cat_pipe__native-country
0,1.134053,0.693530,-0.505302,-1.113327,-0.248381,-0.24026
1,-0.828883,-1.441899,-0.505302,-1.113327,-1.553733,-0.24026
2,1.134053,0.693530,-0.505302,0.354661,1.056971,-0.24026
3,1.526640,0.693530,3.182310,0.354661,0.404295,-0.24026
4,0.348879,0.693530,-0.505302,-1.113327,-0.248381,-0.24026


In [25]:
X_test.head()

,num_pipe__educational-num,num_pipe__gender,cat_pipe__workclass,cat_pipe__marital-status,cat_pipe__occupation,cat_pipe__native-country
0,0.741466,0.693530,-0.505302,0.354661,0.404295,-0.24026
1,-0.436295,0.693530,-0.505302,-1.113327,0.077957,-0.24026
2,-0.043708,0.693530,-0.505302,0.354661,-0.248381,-0.24026
3,-1.221470,-1.441899,-0.505302,-1.113327,1.056971,-0.24026
4,-2.399231,0.693530,-0.505302,-1.113327,2.035985,-0.24026


In [26]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
#from sklearn.metrics import r2_score, root_mean_squared_error, mean_absolute_error, mean_squared_error

In [27]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error



In [28]:
regression = LinearRegression()
regression.fit(X_train, y_train)

LinearRegression()

In [29]:
regression.coef_

array([0.1374224 , 0.08514736, 0.0245196 , 0.06464359, 0.01234314,
       0.00189918])

In [30]:
regression.intercept_

0.24831937728505726

In [31]:
import numpy as np
def model_evaluation(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [32]:
models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    mae, rmse, r2_square=model_evaluation(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('*'*90)
    print('\n')

LinearRegression
Model Training Performance
RMSE: 0.389056096919959
MAE: 0.3203980760986144
R2 score 18.487970480990214
******************************************************************************************


Lasso
Model Training Performance
RMSE: 0.43092887993770074
MAE: 0.37235656371417986
R2 score -0.0019472017338761916
******************************************************************************************


Ridge
Model Training Performance
RMSE: 0.38905605833190793
MAE: 0.32039805376408903
R2 score 18.48798665033048
******************************************************************************************


Elasticnet
Model Training Performance
RMSE: 0.43092887993770074
MAE: 0.37235656371417986
R2 score -0.0019472017338761916
******************************************************************************************




In [33]:
model_list

['LinearRegression', 'Lasso', 'Ridge', 'Elasticnet']